In this blog post, we will investigate the Seaborn Attention dataset. Specifically, we will focus on using the Naive Bayes classifier to predict whether a test-taker is focused or not depending on their score. 

# Get and Examine the Data

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

sns.get_dataset_names()
attention_raw = sns.load_dataset('attention')
attention_raw.head()

In [ ]:
attention_raw.info()

Let's drop the Unnamed column, since it will not help with our analysis. 

In [ ]:
attention = attention_raw.drop("Unnamed: 0", axis=1)
attention.head()

Let's explore the dataset and see if there are any missing/outlier values. 

In [ ]:
attention.isna().any().any()

In [ ]:
attention.shape

In [ ]:
[attention['subject'].min(), attention['subject'].max()]

In [ ]:
[attention['solutions'].min(), attention['solutions'].max()]

In [ ]:
[attention['score'].min(), attention['score'].max()]

There are 20 subjects total, who were given 3 different tests. For each of these tests, the score of the subjects was computed. The minimum score across all tests was 2.0, while the maximum score was 9.0.

# Visualizations

Let's now visualize the attention dataset. We will first display a kde plot, that shows the probability distribution of the scores for the focused and divided groups. 

In [ ]:
sns.displot(data=attention, x='score', kde=True, hue='attention', stat='density');

As can be seen by this plot, the probability that a divided participant performed very well (i.e. got a score of 7 or higher) is substantially less than the probability that a focused user performed very well. Also, the probability that focused users performed poorly (i.e. got a score of 4 or lower) is substantially smaller than the probability that distracted users performed poorly. 

It appears that focused users, on average, received better scores than non-focused users. Let's try to better visualize this trend by plotting subjects and their corresponding scores, and color-coding whether the subject was focused or not. 

In [ ]:
sns.barplot(data=attention, x='subject', y='score', hue='attention');

It appears that, on average, focused subjects appeared to perform better than non-focused subjects. Let's confirm this hypothesis by computing the average scores for the divided and focused groups. 

In [ ]:
avg_scores = pd.DataFrame(attention.groupby('attention').mean()['score'])
avg_scores

Indeed, we can see that the average score for divided users is around 5, while the average score for focused users is approximately 7. It appears that the average test score for focused users is about 2 points higher than that of distracted users.  


# Naive Bayes Theory

Before getting hands-on experience using the Naive Bayes Classifier, it will be useful to explain the theory underlying this classifier. 

Naive Bayes is based on Bayes' Rule, which can be stated as follows: 

$$\huge{P(A|B) = \frac{P(A) P(B|A)}{P(B)}}$$

Where  
P(A|B) is the probability that event A occurs given that event B has already occurred.  
P(A) is the probability of event A occurring.  
P(B|A) is the probability that event B occurs given that event A has already occurred.  
P(B) is the probability of event B occurring. 

A Naive Bayes Classifier is designed to classify a given observation into one of several classes. In order to do so, the Naive Bayes Classifier relies heavily on Bayes theorem. Specifically, given a specific feature of the dataset, the Naive Bayes Classifier computes the probability of the observation falling into each of the different classes based on the value of the feature. Then, this classifier takes the highest such probability, and classifies the observation as falling into the class which matches with that largest probability. 

One important consideration is that the Naive Bayes classifier assumes that features are conditionally independent of one another. This means that when using multiple features in order to predict which class an observation falls into, the Naive Bayes Classifier assumes that none of the features depend on each other. 

# Naive Bayes Classifier

Let's now use a Naive Bayes classifier to predict whether a participant is distracted or not based on their score. We can drop the other columns, since they won't be relevant. 

In [ ]:
attention_nb = attention[['attention', 'score']]
attention_nb.head()

Next, perform a train-test split. Since the dataset is small, let's choose 80% of the samples to be in the training set and 20% of the samples to be in the test set. 

In [ ]:
from sklearn.model_selection import train_test_split

X = attention_nb[['score']]
y = attention_nb['attention']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
X_train.shape
X_test.shape
y_train.shape
y_test.shape

Now, let's instantiate the model and fit the data to the model. We will use a Gaussian Naive Bayes classifier, which assumes that the distribution of continuous features is Gaussian. This appears to be a reasonable assumption in the case of the test scores. 

In [ ]:
from sklearn.naive_bayes import GaussianNB

nb_gauss = GaussianNB()
nb_gauss.fit(X_train, y_train);

Let's evaluate the accuracy on our testing set. 

In [ ]:
predictions = nb_gauss.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score as accuracy
round(accuracy(y_test, predictions), 2)

In [ ]:
predictions

75% is actually quite good, considering how few samples were used to train the model. We can plot a confusion matrix to get a better sense for which samples the model misclassified. 

In [ ]:
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay

labels_arr = ['divided', 'focused']
conf_matrix = metrics.confusion_matrix(y_test, predictions, labels=labels_arr)
cm = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=['divided', 'focused'])
cm.plot();

It appears that the Gaussian NB classifier over-predicted samples as being focused. The three samples in the upper right quadrant were predicted as being focused, but were in fact divided. 

# Attributions

1. https://seaborn.pydata.org/generated/seaborn.barplot.html
2. https://www.geeksforgeeks.org/python-pandas-dataframe-groupby/
3. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html
4. https://scikit-learn.org/stable/modules/naive_bayes.html
5. https://www.datacamp.com/tutorial/naive-bayes-scikit-learn
6. https://corporatefinanceinstitute.com/resources/data-science/bayes-theorem/
7. https://www.bayesrulesbook.com/chapter-14
